In [2]:
#1.1 - IMPORT THE NECESSARY LIBRARIES
import os
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree
from sklearn.neighbors import LocalOutlierFactor
from numba.typed import List
from scipy.spatial import cKDTree
from numba import njit, prange
import itertools

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
print("Python version:", sys.version)
print("NumPy version:", np.__version__)
print("Open3D version:", o3d.__version__)
print("Plotly version:", plotly.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Scikit-learn version:", sklearn_version)
print("SciPy version:", scipy_version)
print("Numba version:", numba_version)

Python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
NumPy version: 1.23.2
Open3D version: 0.17.0
Plotly version: 5.14.1
Matplotlib version: 3.7.1
Scikit-learn version: 1.2.2
SciPy version: 1.9.1
Numba version: 0.57.0


In [1]:
print("jo")

jo


In [ ]:
def get_fileNames(folder_path):
    
    # create an empty list to store the file names
    file_names = []

    # go through each file in the folder
    for filename in os.listdir(folder_path):
        # add the file name to the list
        file_names.append(filename)

    # print the list of file names
    return(file_names)

In [ ]:
def remove_outliers_lof(points, k=20, contamination=0.05):
    # Create an instance of the LocalOutlierFactor class with the specified parameters
    lof = LocalOutlierFactor(n_neighbors=k, contamination=contamination)

    # Calculate the LOF scores for each point in the dataset
    lof_scores = lof.fit_predict(points)

    # Select the inliers from the dataset based on their LOF scores
    # Points with LOF scores less than or equal to 1 are considered inliers
    inliers = points[lof_scores == 1]

    # Return the filtered array of inliers
    return inliers


In [ ]:
def get_avg_point_distance(pcd):
    # Compute the nearest neighbor distance for each point in the point cloud
    distances = pcd.compute_nearest_neighbor_distance()

    # Compute the average nearest neighbor distance for all points in the point cloud
    avg_distance = np.mean(distances)

    # Return the average distance as the result of this function
    return avg_distance

def get_best_distance_threshold(pcd, scale_factor=1.5):
    # Get the average point-to-point distance for the point cloud
    avg_distance = get_avg_point_distance(pcd)

    # Compute the best distance threshold based on the average distance and scale factor
    threshold = scale_factor * avg_distance

    # Return the computed threshold as the result of this function
    return threshold


In [ ]:
def estimate_radius(points, k=16):
    """
    Estimate the radius of a point cloud using the k-Nearest Neighbors method.

    Parameters:
    points (numpy.ndarray): A numpy array containing the point cloud data.
    k (int): The number of nearest neighbors to consider.

    Returns:
    float: The estimated radius.
    """

    # Fit a k-NN model to the data using the specified number of neighbors
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(points)

    # Find the distances to the k nearest neighbors for each point in the dataset
    distances, _ = nbrs.kneighbors(points)

    # Compute the average distance to the k nearest neighbors for each point
    avg_distances = np.mean(distances, axis=1)

    # Estimate the radius as the median of the average distances
    radius = np.median(avg_distances)

    # Return the estimated radius
    return radius


In [ ]:
def draw_plotly(rest):
    """
    Visualize a point cloud using Plotly.

    Parameters:
    rest (open3d.geometry.PointCloud): The point cloud to visualize.

    Returns:
    None.
    """

    # Convert the point cloud to a numpy array
    points = np.asarray(rest.points)

    # Create a new Plotly figure
    fig = go.Figure()

    # Add a scatter plot of the main point cloud
    if rest.colors != None:
        # If the point cloud has color information, use it for the scatter plot
        colors = np.asarray(rest.colors)
    else:
        # If there is no color information, use a default color
        colors = np.ones((len(points), 3))

    fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2],
                               mode='markers',
                               marker=dict(size=1, color=colors, colorscale='Viridis', showscale=False)))

    # Set the aspect ratio of the plot and show it
    fig.update_layout(scene=dict(aspectmode='data'))
    fig.show()

In [ ]:
# def draw_plotly(rest):
#     """
#     Visualize a point cloud and its segments using Plotly.

#     Parameters:
#     rest (open3d.geometry.PointCloud): The point cloud to visualize.
#     segments (List[open3d.geometry.PointCloud]): A list of point clouds representing the segments.

#     Returns:
#     None.
#     """

#     # Convert the point cloud to a numpy array
#     points = np.asarray(rest.points)

#     # Create a new Plotly figure
#     fig = go.Figure()

#     # Add a scatter plot of the main point cloud
#     if rest.colors != None:
#         # If the point cloud has color information, use it for the scatter plot
#         colors = np.asarray(rest.colors)
#     else:
#         # If there is no color information, use a default color
#         colors = np.ones(len(np.asarray(rest.points)))

#     fig.add_traces(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], 
#             mode='markers',
#             marker=dict(size=1, color=colors)))

# #     # Add a scatter plot for each segment
# #     for i in range(len(segments)):
# #         pc = np.asarray(segments[i].points)
# #         if segments[i].colors != None:
# #             # If the segment has color information, use it for the scatter plot
# #             colors = np.asarray(segments[i].colors)
# #         else:
# #             # If there is no color information, use a default color
# #             colors = np.ones(len(np.asarray(segments[i].points)))
# #         fig.add_traces(go.Scatter3d(x=pc[:,0], y=pc[:,1], z=pc[:,2], 
# #             mode='markers',
# #             marker=dict(size=1, color=colors)))

#     # Set the aspect ratio of the plot and show it
#     fig.update_layout(dict(scene=dict(aspectmode='data')))
#     fig.show()


In [ ]:
def subtract_and_add_point_clouds(pcd_A: o3d.geometry.PointCloud, pcd_B: o3d.geometry.PointCloud, voxel_size: float = 0.01, threshold: float = 1e-3) -> tuple:
    """
    Remove the points in point cloud A that are close to point cloud B and add them to point cloud B.

    Parameters:
    pcd_A (open3d.geometry.PointCloud): The first point cloud.
    pcd_B (open3d.geometry.PointCloud): The second point cloud.
    voxel_size (float): The voxel size for downsampling (default 0.01).
    threshold (float): The threshold distance for determining if a point in A is close to B (default 1e-3).

    Returns:
    tuple: A tuple containing the cleaned point cloud A and the updated point cloud B.
    """
    # Perform voxel downsampling on both point clouds
    downsampled_pcd_A = pcd_A.voxel_down_sample(voxel_size)
    downsampled_pcd_B = pcd_B.voxel_down_sample(voxel_size)

    # Create a KDTree for the downsampled point cloud B
    kdtree = o3d.geometry.KDTreeFlann(downsampled_pcd_B)

    # Get the points from the point clouds as numpy arrays
    points_A = np.asarray(downsampled_pcd_A.points)
    points_B = np.asarray(downsampled_pcd_B.points)

    # Initialize an array to store the indices of points to be removed
    indices_to_remove = []

    # Iterate through the points in the downsampled point cloud A
    for i, point in enumerate(points_A):
        # Find the nearest neighbor in the downsampled point cloud B
        _, idx, _ = kdtree.search_knn_vector_3d(point, 3)

        # Check if the distance is less than the threshold
        if np.linalg.norm(points_B[idx[0]] - point) < threshold:
            indices_to_remove.append(i)

    # Remove the marked points from the downsampled point cloud A
    cleaned_points_A = np.delete(points_A, indices_to_remove, axis=0)

    # Add the removed points to point cloud B
    removed_points_A = points_A[indices_to_remove]
    updated_points_B = np.vstack((points_B, removed_points_A))

    # Create new point clouds with the cleaned and updated points
    cleaned_pcd_A = o3d.geometry.PointCloud()
    cleaned_pcd_A.points = o3d.utility.Vector3dVector(cleaned_points_A)
    
    

    updated_pcd_B = o3d.geometry.PointCloud()
    updated_pcd_B.points = o3d.utility.Vector3dVector(updated_points_B)
    
    return cleaned_pcd_A, updated_pcd_B


In [ ]:
def compute_segment_distances(segments, threshold):
    # Get the keys of the segments dictionary
    segment_keys = list(segments.keys())

    # Get the number of segments
    n = len(segment_keys)

    # Initialize a 2D array to store the distances between segments
    distances = np.zeros((n, n))

    # Iterate through all combinations of segments
    for i, j in itertools.combinations(range(n), 2):
        # Compute the distances between shared points of the two segments
        shared_points = segments[segment_keys[i]].compute_point_cloud_distance(segments[segment_keys[j]])

        # Count the number of shared points that are within the threshold distance
        num_shared_points = np.sum(np.array(shared_points) < threshold)

        # Store the number of shared points as the distance between the segments
        distances[i, j] = num_shared_points
        distances[j, i] = num_shared_points

    return distances


def assign_colors_to_small_segments(segments, pcd, min_points, threshold):
    # Compute the distances between all pairs of segments
    segment_distances = compute_segment_distances(segments, threshold)

    # Initialize a new dictionary to store the updated segments
    new_segments = {}

    # Iterate through all segments
    for i, segment in segments.items():
        # Check if the segment has less points than the minimum threshold
        if len(segment.points) < min_points:
            # Find the neighboring segment with the most points
            max_points = 0
            max_points_segment_idx = None
            for j, neighbor_segment in segments.items():
                # Skip the current segment
                if i == j:
                    continue

                # Get the number of shared points with the current segment
                num_shared_points = segment_distances[i, j]

                # Update the index of the neighboring segment with the most points
                if num_shared_points > max_points:
                    max_points = num_shared_points
                    max_points_segment_idx = j

            if max_points_segment_idx is not None:
                # Assign the color of the neighboring segment with the most points to the current segment
                segment.paint_uniform_color(segments[max_points_segment_idx].colors[0])

        # Add the updated or unchanged segment to the new dictionary
        new_segments[i] = segment

    return new_segments

In [ ]:
def refined_ransac(pcd):
    segment_models = {}  # dictionary to store the models of segmented planes
    segments = {}  # dictionary to store the segmented planes
    max_plane_idx = 40  # maximum number of planes to segment
    rest = pcd  # remaining point cloud to segment after each iteration
    no_ = len(np.array(rest.points))  # number of points in the original point cloud
    for i in range(max_plane_idx):
        if len(rest.points) < 0.010*len(np.array(pcd.points)):  # check if there are enough points left to segment
            print("Not enough points left for RANSAC. Stopping at iteration:", i)
            break
        d_threshold = get_best_distance_threshold(rest)  # get the best distance threshold for plane segmentation
        # segment the plane and get the inliers
        segment_models[i], inliers = rest.segment_plane(distance_threshold=d_threshold, ransac_n=3, num_iterations=1000)
        segments[i] = rest.select_by_index(inliers)  # select the points in the plane segment
        # cluster the segment points and get the cluster with the most points
        labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
        candidates = [len(np.where(labels == j)[0]) for j in np.unique(labels)]
        best_candidate = int(np.array(np.unique(labels)[np.where(candidates == np.max(candidates))[0]])[0])
        print("the best candidate is:", best_candidate)
        # update the remaining point cloud and segment points based on the best cluster
        rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels != best_candidate)[0]))
        segments[i] = segments[i].select_by_index(list(np.where(labels == best_candidate)[0]))
        random_color = np.random.rand(3).tolist()  # Generate a random color
        segments[i].paint_uniform_color(random_color)  # Assign the random color to the segment
        print("pass", i + 1, "/", max_plane_idx, "done.")
    return rest, segments, i, segment_models


In [ ]:
@njit
def calc_curvature(neighbors):
    # If there are fewer than 2 neighbors, return 0 curvature
    if neighbors.shape[0] < 2:
        return 0
    # Calculate the center of the neighbors
    center = np.sum(neighbors, axis=0) / neighbors.shape[0]
    # Compute the covariance matrix
    cov = np.cov(neighbors.T)
    # Perform singular value decomposition (SVD) on the covariance matrix
    _, s, _ = np.linalg.svd(cov)
    # Calculate and return the curvature
    return s[-1] / np.sum(s)

@njit(parallel=True)
def compute_curvature(points, indices):
    # Initialize an array to store curvature values
    curvature = np.zeros(len(points))

    # Calculate curvature for each point in the point cloud
    for i in prange(len(points)):
        idx = indices[i]
        neighbors = points[idx, :]
        curvature[i] = calc_curvature(neighbors)

    # Return the array of curvature values
    return curvature

def calculate_curvature_threshold(curvature, percentile=95):
    # Calculate and return the curvature threshold based on the given percentile
    return np.percentile(curvature, percentile)

In [ ]:
# Get file names from a directory
fileNames = get_fileNames("Archive")

# Choose a file to read (in this case, the fifth file in the directory)
current_file = 4

# Read a triangle mesh from the chosen file
mesh = o3d.io.read_triangle_mesh('Archive'+'/'+fileNames[current_file])  

# Sample points uniformly from the mesh to get a point cloud
pcd = mesh.sample_points_uniformly(number_of_points=100_000)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [ ]:
#To compute the curvature and the points that lie on the curvature

# Convert point cloud data to a NumPy array
points = np.asarray(pcd.points)
# Estimate the radius for neighborhood search
radius = estimate_radius(points)
# Build a k-d tree for efficient nearest neighbor search
kdtree = cKDTree(points)
# Query the k-d tree to find the k nearest neighbors within the specified radius
_, indices = kdtree.query(points, k=12, distance_upper_bound=radius)
# Remove any invalid indices from the result
indices = [idx[idx != kdtree.n] for idx in indices]

# Create a numba-compatible jagged array for the indices
indices_jagged = List()
for idx in indices:
    indices_jagged.append(np.asarray(idx))

# Compute the curvature for each point in the point cloud using the jagged array of indices
curvature = compute_curvature(points, indices_jagged)


#VERY IMPORTANT - PLEASE VISUALIZE BEFOREHAND TO SEE IF THE PERCENTILE THRESHOLD NEEDS TO BE ALTERED

In [ ]:
#VERY IMPORTANT - PLEASE VISUALIZE BEFOREHAND TO SEE IF THE PERCENTILE THRESHOLD NEEDS TO BE ALTERED
# Increase the value if there are any noise points added in the high_curvature_pcd added and you would want to remove them
# Decrease the value if you want to increase the number of points to be included in the high_curvature_pcd
percentile = 53

curvature_threshold = calculate_curvature_threshold(curvature, percentile)

# Select points with high curvature
high_curvature_points = points[curvature > curvature_threshold]

# Visualize the high curvature points with a different color
high_curvature_pcd = o3d.geometry.PointCloud()
high_curvature_pcd.points = o3d.utility.Vector3dVector(high_curvature_points)

o3d.visualization.draw_geometries([high_curvature_pcd])
print(curvature_threshold)

In [ ]:
#Removing the points that lie on the curvature from the original point cloud and adding duplicate points 
pcd_A = pcd
pcd_B = high_curvature_pcd

resultant_pcd, high_curvature_pcd = subtract_and_add_point_clouds(pcd_A, pcd_B, voxel_size=0.01, threshold=1.3)
high_curvature_pcd.paint_uniform_color([0, 0, 0])  # black color for high curvature points
# o3d.visualization.draw_geometries([high_curvature_pcd])

In [ ]:
# o3d.visualization.draw_geometries([resultant_pcd])

In [ ]:
points = np.asarray(resultant_pcd.points)

# Remove outliers using LOF
k = 5  # Number of neighbors to consider
contamination = 0.01  # Proportion of outliers in the data set
inliers = remove_outliers_lof(points, k, contamination)

# Visualize the point cloud after removing outliers
inliers_pcd = o3d.geometry.PointCloud()
inliers_pcd.points = o3d.utility.Vector3dVector(inliers)
# o3d.visualization.draw_geometries([inliers_pcd])


In [ ]:
rest, segments, max_plane_idx, models = refined_ransac(inliers_pcd)
rest.paint_uniform_color([0, 0, 0]) 

In [ ]:
# o3d.visualization.draw_geometries([rest])

In [ ]:
a = o3d.geometry.PointCloud()
for i in range(len(segments)):
    a = a + segments[i]
 

In [ ]:
o3d.visualization.draw_geometries([a+rest+high_curvature_pcd])

In [ ]:
min_points = int(0.35 * len(np.array(pcd.points)))  # Set this to the desired minimum number of points for each segment
updated_segments = assign_colors_to_small_segments(segments, pcd, min_points, get_best_distance_threshold(pcd))
a = o3d.geometry.PointCloud()
for i in range(len(updated_segments)):
    a = a + updated_segments[i]
# o3d.visualization.draw_geometries([a])

In [ ]:
o3d.visualization.draw_geometries([a+high_curvature_pcd+rest]) #final visualization

In [ ]:
final_pcd = a+rest+high_curvature_pcd

In [ ]:
downsampled_pcd = final_pcd.voxel_down_sample(0.5)
downsampled_pcd
draw_plotly([downsampled_pcd])

In [ ]:
def unique_colors(pcd: o3d.geometry.PointCloud) -> np.ndarray:
    # Extract colors from the point cloud and convert to a numpy array
    colors = np.asarray(pcd.colors)

    # Create a structured array view with a single field, then use numpy's unique function
    dtype = {'names': ['color'], 'formats': [colors.dtype] * colors.shape[1]}
    unique_colors = np.unique(colors.view(dtype))

    # Convert the structured array back to the original format
    return unique_colors.view(colors.dtype).reshape(-1, colors.shape[1])

# Load or create a point cloud (pcd)
# pcd = o3d.io.read_point_cloud("path/to/your/pointcloud.ply")

# Find unique colors in the point cloud
unique_colors_array = unique_colors(a)
print(unique_colors_array) #colors representing segments
print("number of segments:",len(unique_colors_array)) #total number of segments excluding from points on the curvature